# Tavily Extract

[Tavily](https://tavily.com) is a search engine built specifically for AI agents (LLMs), delivering real-time, accurate, and factual results at speed. Tavily offers an [Extract](https://docs.tavily.com/api-reference/endpoint/extract) endpoint that can be used to extract content from a URLs.


## Overview

### Integration details
| Class                                                         | Package                                                        | Serializable | [JS support](https://js.langchain.com/docs/integrations/tools/tavily_search) |  Package latest |
|:--------------------------------------------------------------|:---------------------------------------------------------------| :---: | :---: | :---: |
| [TavilyExtract](https://github.com/tavily-ai/langchain-tavily) | [langchain-tavily](https://pypi.org/project/langchain-tavily/) | ❌ | ❌  |  ![PyPI - Version](https://img.shields.io/pypi/v/langchain-tavily?style=flat-square&label=%20) |

### Tool features
| [Returns artifact](/docs/how_to/tool_artifacts/) | Native async |                       Return data                        | Pricing |
| :---: | :---: |:--------------------------------------------------------:| :---: |
| ❌ | ✅ | raw content and images | 1,000 free searches / month |


## Setup

The integration lives in the `langchain-tavily` package.

In [1]:
%pip install -qU langchain-tavily

Note: you may need to restart the kernel to use updated packages.


### Credentials

We also need to set our Tavily API key. You can get an API key by visiting [this site](https://app.tavily.com/sign-in) and creating an account.

In [2]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

## Instantiation


The tool accepts various parameters during instantiation:

- `extract_depth` (optional, str): The depth of the extraction, either "basic" or "advanced". Default is "basic ".
- `include_images` (optional, bool): Whether to include images in the extraction. Default is False.

For a comprehensive overview of the available parameters, refer to the [Tavily Extract API documentation](https://docs.tavily.com/documentation/api-reference/endpoint/extract)

In [3]:
from langchain_tavily import TavilyExtract

tool = TavilyExtract(
    extract_depth="basic",
    include_images=False,
)

## Invocation

### [Invoke directly with args](/docs/concepts/tools)


The Tavily extract tool accepts the following arguments during invocation:
- `urls` (required): A list of URLs to extract content from. 
- Both `extract_depth` and `include_images` can also be set during invokation

NOTE: The optional arguments are available for agents to dynamically set, if you set a argument during instantiation and then invoke the tool with a different value, the tool will use the value you passed during invokation. 

In [4]:
tool.invoke({"urls": ["https://en.wikipedia.org/wiki/Lionel_Messi"]})

{'results': [{'url': 'https://en.wikipedia.org/wiki/Lionel_Messi',
   'raw_content': 'Lionel Messi - Wikipedia\nJump to content\nMain menu\nMain menu\nmove to sidebar hide\nNavigation\n\nMain page\nContents\nCurrent events\nRandom article\nAbout Wikipedia\nContact us\n\nContribute\n\nHelp\nLearn to edit\nCommunity portal\nRecent changes\nUpload file\nSpecial pages\n\n  \nSearch\nSearch\nAppearance\n\nDonate\nCreate account\nLog in\n\nPersonal tools\n\nDonate\nCreate account\nLog in\n\nPages for logged out editors learn more\n\nContributions\nTalk\n\nContents\nmove to sidebar hide\n\n(Top)\n\n1 Early life\n\n\n2 Youth careerToggle Youth career subsection\n\n\n2.1 Newell\'s Old Boys\n\n\n2.2 Barcelona\n\n\n\n\n3 Club careerToggle Club career subsection\n\n\n3.1 Barcelona\n\n\n3.1.1 2004–2008: Rise to the first team\n\n\n3.1.2 2008–2012: Success under Pep Guardiola\n\n\n3.1.3 2012–2014: Record-breaking year and Messidependencia\n\n\n3.1.4 2014–2017: Arrival of Luis Enrique and birth of MS

### [Invoke with ToolCall](/docs/concepts/tools)

We can also invoke the tool with a model-generated ToolCall, in which case a ToolMessage will be returned:

In [5]:
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {"urls": ["https://en.wikipedia.org/wiki/Lionel_Messi"]},
    "id": "1",
    "name": "tavily",
    "type": "tool_call",
}
tool_msg = tool.invoke(model_generated_tool_call)

# The content is a JSON string of results
print(tool_msg.content[:400])

{"results": [{"url": "https://en.wikipedia.org/wiki/Lionel_Messi", "raw_content": "Lionel Messi - Wikipedia\nJump to content\nMain menu\nMain menu\nmove to sidebar hide\nNavigation\n\nMain page\nContents\nCurrent events\nRandom article\nAbout Wikipedia\nContact us\n\nContribute\n\nHelp\nLearn to edit\nCommunity portal\nRecent changes\nUpload file\nSpecial pages\n\n  \nSearch\nSearch\nAppearance\n\


## Use within an agent

We can use our tools directly with an agent executor by binding the tool to the agent. This gives the agent the ability to dynamically set the available arguments to the Tavily search tool.

In the below example when we ask the agent to find "What is the most popular sport in the world? include only wikipedia sources" the agent will dynamically set the argments and invoke Tavily search tool : Invoking `tavily_search` with `{'query': 'most popular sport in the world', 'include_domains': ['wikipedia.org']`

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

In [6]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY:\n")

In [7]:
# | output: false
# | echo: false

# !pip install -qU langchain langchain-openai
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-4o", model_provider="openai", temperature=0)

In [8]:
import datetime

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.schema import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_tavily import TavilyExtract

# Initialize LLM
llm = init_chat_model(model="gpt-4o", model_provider="openai", temperature=0)

# Initialize Tavily Search Tool
tavily_search_tool = TavilyExtract()

# Set up Prompt with 'agent_scratchpad'
today = datetime.datetime.today().strftime("%D")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"""You are a helpful reaserch assistant, you will be given a list of urls. 
            You must extract content from the urls and write a summary of the findings The date today is {today}.""",
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(
            variable_name="agent_scratchpad"
        ),  # Required for tool calls
    ]
)

# Create an agent that can use tools
agent = create_openai_tools_agent(llm=llm, tools=[tavily_search_tool], prompt=prompt)

# Create an Agent Executor to handle tool execution
agent_executor = AgentExecutor(agent=agent, tools=[tavily_search_tool], verbose=True)

user_input = "['https://en.wikipedia.org/wiki/Albert_Einstein','https://en.wikipedia.org/wiki/Theoretical_physics']"

# Construct input properly as a dictionary
response = agent_executor.invoke({"messages": [HumanMessage(content=user_input)]})
response["output"]



> Entering new AgentExecutor chain...

Invoking: `tavily_extract` with `{'urls': ['https://en.wikipedia.org/wiki/Albert_Einstein'], 'extract_depth': 'advanced', 'include_images': False}`


{'results': [{'url': 'https://en.wikipedia.org/wiki/Albert_Einstein', 'raw_content': 'Published Time: 2001-11-05T18:26:16Z\nJump to content\nMain menu\nSearch\nAppearance\nDonate\nCreate account\nLog in\nPersonal tools\n        Photograph your local culture, help Wikipedia and win!\nToggle the table of contents\nAlbert Einstein\n227 languages\nArticle\nTalk\nRead\nView source\nView history\nTools\nFrom Wikipedia, the free encyclopedia\n"Einstein" redirects here. For other uses, see Einstein (disambiguation) and Albert Einstein (disambiguation).\nAlbert Einstein\nEinstein in 1947\nBorn    14 March 1879\nUlm, Kingdom of Württemberg, German Empire\nDied    18 April 1955 (aged\xa076)\nPrinceton, New Jersey, U.S.\nCitizenship \nshow\nSee list\nEducation \nSwiss federal polytechnic school in Zurich (teac

'Here is a summary of the findings from the two Wikipedia articles:\n\n1. **Albert Einstein**:\n   - Albert Einstein was a German-born theoretical physicist renowned for developing the theory of relativity, one of the two pillars of modern physics, alongside quantum mechanics. His mass-energy equivalence formula, E=mc², is one of the most famous equations in the world.\n   - Einstein was awarded the 1921 Nobel Prize in Physics for his explanation of the photoelectric effect, which was pivotal in establishing quantum theory.\n   - He was born on March 14, 1879, in Ulm, in the Kingdom of Württemberg in the German Empire, and died on April 18, 1955, in Princeton, New Jersey, USA.\n   - Throughout his career, Einstein made significant contributions to the development of statistical mechanics, quantum theory, and cosmology. He also worked on a unified field theory, although it was not successful.\n   - Einstein was a prominent figure in the scientific community and was involved in various s

## API reference

For detailed documentation of all Tavily Search API features and configurations head to the API reference: https://docs.tavily.com/documentation/api-reference/endpoint/extract